In [2]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [3]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt

In [164]:
df = pd.read_csv('../Data/ABIDE_df2.csv')
df = df.iloc[~pd.isna(df['bids_folder']).values]
df = df.sort_values(by='bids_folder')
print(len(df))
df.head(3)

1048


,participant_id,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,...,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,bids_folder,site
460,50642,1,1.0,33.0,1,R,NaN,103.0,98.0,107.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-CMUa0050642,1
464,50646,1,1.0,21.0,1,R,NaN,108.0,100.0,115.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-CMUa0050646,1
465,50647,1,1.0,27.0,1,R,NaN,104.0,97.0,109.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-CMUa0050647,1


In [15]:
# data_dir = '../Assets/abide_1_cmats'

In [16]:
# cData = np.array([np.load(os.path.join(data_dir,sub+'.npy')) for sub in df['bids_folder'].values])

# print(f'Original data range: {(cData.min(),cData.max())}')
# cData = (cData+1)/2
# print(f'Normalized data range: {(cData.min(),cData.max())}')
# cData.shape

In [17]:
from tensorflow.keras.layers import ZeroPadding2D
pad2d = ZeroPadding2D(padding=((6,7),(6,7))) #If tuple of 2 tuples of 2 ints: interpreted as ((top_pad, bottom_pad

In [18]:
# cData_pad = pad2d(cData[:,:,:,np.newaxis])
# cData_pad.shape

In [19]:
# plt.imshow(cData_pad[0,:,:,0]);plt.xticks([]);plt.yticks([]);

## DEFINE MODEL AND LOAD WEIGHTS

In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import regularizers
import os
import pandas as pd

def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
      args (tensor): mean and log of variance of Q(z|X)
    # Returns:
      z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def get_fMRI_CVAE_3D(input_shape=(51,51,1),
                     latent_dim=[2,2],
                     beta=1,
                     disentangle=False,
                     gamma=1,
                     bias=True,
                     batch_size = 32,
                     kernel_size = 3,
                     filters = 16,
                     intermediate_dim = 128,
                     nlayers = 2,
                     learning_rate=0.001,
                     opt=None):
    
    
    ndim_bg = latent_dim[0]
    ndim_sl = latent_dim[1]
    
    image_size, _, channels = input_shape

    kernel_regularizer=regularizers.l2(.0001)

    # build encoder model
    tg_inputs = Input(shape=input_shape, name='tg_inputs')
    bg_inputs = Input(shape=input_shape, name='bg_inputs')
    
    BatchNorm = tf.keras.layers.BatchNormalization(
    axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,
    beta_initializer='zeros', gamma_initializer='ones',
    moving_mean_initializer='zeros',
    moving_variance_initializer='ones', beta_regularizer=None,
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

    #kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0,stddev=5)
    kernel_initializer = tf.keras.initializers.RandomUniform()

    # generate latent vector Q(z|X)
    
    
    z_h_layer = Dense(intermediate_dim,activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_mean_layer = Dense(ndim_bg, name='z_mean', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_log_var_layer = Dense(ndim_bg, name='z_log_var', use_bias=bias,kernel_regularizer=kernel_regularizer)
    z_layer = Lambda(sampling, output_shape=(ndim_bg,), name='z')

    def z_encoder_func(inputs):
        z_h = inputs

        these_filters = filters
        for i in range(nlayers):
            these_filters *= 2
            #print(these_filters)
            z_h = Conv2D(filters=these_filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    padding='same',
                    use_bias=bias,
                    kernel_regularizer=kernel_regularizer)(z_h)
        
        # shape info needed to build decoder model
        shape = K.int_shape(z_h)
        z_h = Flatten()(z_h)
        z_h = z_h_layer(z_h)
        z_mean =  z_mean_layer(z_h)
        #z_mean = BatchNorm(z_mean)
        
        z_log_var =  z_log_var_layer(z_h)
        z = z_layer([z_mean, z_log_var])
        return z_mean, z_log_var, z, shape

    tg_z_mean, tg_z_log_var, tg_z, shape_z = z_encoder_func(tg_inputs)

    # generate latent vector Q(z|X)
    s_h_layer = Dense(intermediate_dim, activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_mean_layer = Dense(ndim_sl, name='s_mean', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_log_var_layer = Dense(ndim_sl, name='s_log_var', use_bias=bias,kernel_regularizer=kernel_regularizer)
    s_layer = Lambda(sampling, output_shape=(ndim_sl,), name='s')

    def s_encoder_func(inputs):
        s_h = inputs
        these_filters = filters
        for i in range(nlayers):
            these_filters *= 2
            s_h = Conv2D(filters=these_filters,
                    kernel_size=kernel_size,
                    activation='relu',
                    strides=2,
                    use_bias=bias,
                    kernel_regularizer=kernel_regularizer,
                    padding='same')(s_h)
        
        # shape info needed to build decoder model
        shape = K.int_shape(s_h)
        s_h = Flatten()(s_h)
        s_h = s_h_layer(s_h)
        s_mean =  s_mean_layer(s_h)
        #s_mean = BatchNorm(s_mean)
        
        s_log_var =  s_log_var_layer(s_h)        
        s = s_layer([s_mean, s_log_var])
        
        return s_mean, s_log_var, s, shape

    tg_s_mean, tg_s_log_var, tg_s, shape_s = s_encoder_func(tg_inputs)
    bg_z_mean, bg_z_log_var, bg_z, _ = z_encoder_func(bg_inputs) # Aidas and Stefano team hax
    
    
    # instantiate encoder models
    z_encoder = tf.keras.models.Model(tg_inputs, [tg_z_mean, tg_z_log_var, tg_z], name='z_encoder')
    s_encoder = tf.keras.models.Model(tg_inputs, [tg_s_mean, tg_s_log_var, tg_s], name='s_encoder')


    # build decoder model
    latent_inputs = Input(shape=(ndim_bg+ndim_sl,), name='z_sampling')

    x = Dense(intermediate_dim, activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer,kernel_initializer=kernel_initializer)(latent_inputs)
    x = Dense(shape_z[1] * shape_z[2] * shape_z[3], activation='relu', use_bias=bias,kernel_regularizer=kernel_regularizer,kernel_initializer=kernel_initializer)(x)
    x = Reshape((shape_z[1], shape_z[2], shape_z[3]))(x)

    these_filters = filters*(2**nlayers)/2
    for i in range(nlayers-1):
        x = Conv2DTranspose(filters=these_filters,
                          kernel_size=kernel_size,
                          activation='relu',
                          strides=2,
                          use_bias=bias,
                          kernel_regularizer=kernel_regularizer,
                          padding='same')(x)
        these_filters //= 2

    outputs = Conv2DTranspose(filters=channels,
                            kernel_size=kernel_size,
                            activation='sigmoid',
                            padding='same',
                            strides=2,
                            use_bias=bias,
                            kernel_regularizer=kernel_regularizer,
                            name='decoder_output')(x)

    # instantiate decoder model
    cvae_decoder = Model(latent_inputs, outputs, name='decoder')
      # decoder.summary()

    def zeros_like(x):
        return tf.zeros_like(x)

    tg_outputs = cvae_decoder(tf.keras.layers.concatenate([tg_z, tg_s], -1))
    zeros = tf.keras.layers.Lambda(zeros_like)(tg_s)

    bg_outputs = cvae_decoder(tf.keras.layers.concatenate([bg_z, zeros], -1)) # Aidas look into this, is this correct

    cvae = tf.keras.models.Model(inputs=[tg_inputs, bg_inputs], 
                                  outputs=[tg_outputs, bg_outputs],
                                  name='contrastive_vae')

#     cvae_fg = tf.keras.models.Model(inputs=tg_inputs, 
#                                   outputs=fg_outputs, 
#                                   name='contrastive_vae_fg')

    if disentangle:
        discriminator = Dense(1, activation='sigmoid')

        z1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_z)
        z2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_z)
        s1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_s)
        s2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_s)

        q_bar = tf.keras.layers.concatenate(
          [tf.keras.layers.concatenate([s1, z2], axis=1),
          tf.keras.layers.concatenate([s2, z1], axis=1)],
          axis=0)

        q = tf.keras.layers.concatenate(
          [tf.keras.layers.concatenate([s1, z1], axis=1),
          tf.keras.layers.concatenate([s2, z2], axis=1)],
          axis=0)

        q_bar_score = (discriminator(q_bar)+.1) *.85 # +.1 * .85 so that it's 0<x<1
        q_score = (discriminator(q)+.1) *.85 
        tc_loss = K.log(q_score / (1 - q_score)) 
        discriminator_loss = - K.log(q_score) - K.log(1 - q_bar_score)
    else:
        tc_loss = 0
        discriminator_loss = 0


    reconstruction_loss = tf.keras.losses.mse(K.flatten(tg_inputs[:,6:57,6:57,:]), K.flatten(tg_outputs[:,6:57,6:57,:])) 
    reconstruction_loss += tf.keras.losses.mse(K.flatten(bg_inputs[:,6:57,6:57,:]), K.flatten(bg_outputs[:,6:57,6:57,:])) 
    reconstruction_loss *= input_shape[0] * input_shape[1] * input_shape[2]

    kl_loss1 = 1 + tg_z_log_var - tf.keras.backend.square(tg_z_mean) - tf.keras.backend.exp(tg_z_log_var)
    kl_loss2 = 1 + tg_s_log_var - tf.keras.backend.square(tg_s_mean) - tf.keras.backend.exp(tg_s_log_var)
    kl_loss3 = 1 + bg_z_log_var - tf.keras.backend.square(bg_z_mean) - tf.keras.backend.exp(bg_z_log_var)

    kl_loss1 = tf.keras.backend.sum(kl_loss1, axis=-1)
    kl_loss2 = tf.keras.backend.sum(kl_loss2, axis=-1)
    kl_loss3 = tf.keras.backend.sum(kl_loss3, axis=-1)

    kl_loss = kl_loss1+kl_loss2+kl_loss3
    #kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    
    cvae_loss = tf.keras.backend.mean(reconstruction_loss + beta*kl_loss + gamma*tc_loss + discriminator_loss)
    cvae.add_loss(cvae_loss)
    
    if type(opt)==type(None):
        #print('optimizer not specified using ADAM, wroom wroom')
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False,name='Adam')
        #opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-07, centered=False, name='RMSprop')
        #opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.1, nesterov=False, name='SGD')


    cvae.compile(optimizer=opt,run_eagerly=True)
    
    return cvae, z_encoder, s_encoder, cvae_decoder

In [21]:
#import make_models;reload(make_models);from make_models import *

batch_size = 32
cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_3D(input_shape=(64,64,1),
                                                             latent_dim=[16,16],
                                                             beta=.001,
                                                             gamma=1,
                                                             disentangle=True,
                                                             bias=True,
                                                             batch_size = batch_size,
                                                             kernel_size = 3,
                                                             filters = 8,
                                                             intermediate_dim = 128,
                                                             nlayers = 4,
                                                             learning_rate=0.001,
                                                             opt=None)

num_params = np.sum([np.prod(val.get_shape()) for val in cvae.trainable_weights])
print(f'# params| {num_params:,}')

# params| 1,189,730


In [22]:
weights_dir = '../Assets/tf_weights/'
analysis_name = 'CVAE_2021-10-27 11:41:44.052073'
#analysis_name = 'CVAE_2021-10-28 05:12:17.623449'
weights_fn = os.path.join(weights_dir,analysis_name,'cvae_weights')

In [23]:
cvae.load_weights(weights_fn)

## GET DATA

In [44]:
data = np.load('../Data/rel-cmats.npz',allow_pickle=True)
list(data.keys())

['arr', 'subs']

In [45]:
cmats = data['arr']
subs = data['subs']

isnan = np.array([np.isnan(cmats[i,:,:,:]).sum()==0 for i in range(cmats.shape[0])])
cmats = cmats[isnan,:,:,:]
subs = subs[isnan]
cmats.shape

(1086, 51, 51, 2)

In [101]:
nsubs = cmats.shape[0]
svec = np.arange(nsubs)

In [77]:
Z1_mu,Z1_var,Z1 = z_encoder.predict(pad2d(cmats[:,:,:,0][:,:,:,np.newaxis]))
Z2_mu,Z2_var,Z2 = z_encoder.predict(pad2d(cmats[:,:,:,1][:,:,:,np.newaxis]))

S1_mu,S1_var,S1 = z_encoder.predict(pad2d(cmats[:,:,:,0][:,:,:,np.newaxis]))
S2_mu,S2_var,S2 = z_encoder.predict(pad2d(cmats[:,:,:,1][:,:,:,np.newaxis]))

In [271]:
def getRanks(first_half,second_half):
    import scipy
    from tqdm import tqdm
    corr = scipy.stats.spearmanr
    dist = lambda v1,v2 : corr(v1,v2)[0]
    #dist = lambda v1,v2 : scipy.spatial.distance.pdist(np.array([v1,v2]))[0]

    s = 0
    ranks = list()
    for s in tqdm(range(nsubs)):
        within = dist(first_half[s,:],second_half[s,:])
        other_subs = svec[svec!=s]
        across = np.array([dist(first_half[s,:],second_half[ss,:]) for ss in other_subs])
        rank = (within < across).sum()+1
        #rank = (within > across).sum()+1
        ranks.append(rank)

    return ranks

In [272]:
common_subs = np.intersect1d(df['bids_folder'].values,subs)
df = df.iloc[np.array([val in common_subs for val in df['bids_folder'].values])]
common_subs.shape

(1046,)

In [273]:
Z_ranks = getRanks(Z1,Z2)

100%|██████████| 1086/1086 [07:37<00:00,  2.37it/s]


In [274]:
S_ranks = getRanks(S1,S2)

100%|██████████| 1086/1086 [07:37<00:00,  2.37it/s]


In [275]:
from helper_funcs import get_triu
C1 = np.array([get_triu(cmats[i,:,:,0]) for i in range(nsubs)])
C2 = np.array([get_triu(cmats[i,:,:,1]) for i in range(nsubs)])
C_ranks = getRanks(C1,C2)

100%|██████████| 1086/1086 [11:08<00:00,  1.63it/s]


In [276]:
C_ranks = np.array(C_ranks)
Z_ranks = np.array(Z_ranks)
S_ranks = np.array(S_ranks)
S_ranks.shape

(1086,)

In [277]:
vv = np.array([sub in common_subs for sub in subs])
C_rankz = C_ranks[vv]
Z_rankz = Z_ranks[vv]
S_rankz = S_ranks[vv]

In [278]:
print(C_rankz.shape)
print(Z_ranks.shape)
print(S_ranks.shape)

(1046,)
(1086,)
(1086,)


In [279]:
print((C_rankz==1).mean()*100)
print((Z_ranks==1).mean()*100)
print((S_ranks==1).mean()*100)

88.14531548757171
3.1307550644567224
3.2228360957642725


In [280]:
np.median(C_rankz)

1.0

In [206]:
np.median(Z_ranks)

107.5

In [208]:
np.median(S_ranks)

104.5

In [232]:
patients = df['DX_GROUP'].values==1

In [233]:
(C_rankz[patients]==1).mean()

0.904296875

In [234]:
(Z_ranks[patients]==1).mean()

0.03125

In [235]:
(S_ranks[patients]==1).mean()

0.03515625

In [48]:
# patients = df['DX_GROUP'].values==1

In [49]:
# use_data = cData_pad

# Z_mu,Z_sigma,Z = z_encoder.predict(use_data)
# S_mu,S_sigma,S = s_encoder.predict(use_data)

In [50]:
# from tqdm import tqdm

In [51]:
# z_samples = np.array([z_encoder.predict(use_data)[2] for i in tqdm(range(100))])
# s_samples = np.array([s_encoder.predict(use_data)[2] for i in tqdm(range(100))])

In [52]:
# z_samples.shape

In [53]:
# import umap
# reducer = umap.UMAP(n_components=2)

In [54]:
# s_embedding = reducer.fit_transform(S[patients,:])
# z_embedding = reducer.fit_transform(Z[patients,:])

In [55]:
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,2);plt.scatter(z_embedding[:,0],z_embedding[:,1])
# plt.subplot(1,2,1);plt.scatter(s_embedding[:,0],s_embedding[:,1])

In [56]:
# def predict_cmat(s,is_asd,idx=None):
#     # s = 0
#     # idx=None
#     # is_asd = patients[s]
#     # is_asd = False

#     if type(idx)==type(None):
#         bg_vec = Z[s,:]
#         sl_vec = S[s,:]
#     else:
#         bg_vec = z_samples[idx,s,:]
#         sl_vec = s_samples[idx,s,:]

#     zeroes_ = np.zeros(sl_vec.shape)

#     if is_asd:
#         vec = np.hstack((bg_vec,sl_vec))
#     else:
#         vec = np.hstack((bg_vec,zeroes_))

#     predicted = cvae_decoder.predict(vec[np.newaxis,:])[0,:,:,0]
#     predicted_depad = predicted[6:57,6:57]
    
#     return predicted_depad

In [57]:
# predicted = np.array([predict_cmat(s,patients[s],idx=None) for s in tqdm(range(len(df)))])
# predicted_bg = np.array([predict_cmat(s,False,idx=None) for s in tqdm(range(len(df)))])

### Variability of samples

In [58]:
# cc = np.array([predict_cmat(s=0,is_asd=patients[0],idx=i) for i in range(100)])

In [59]:
# c = np.array([get_triu(cc[i,:,:]) for i in range(100)]).std(axis=0).mean()

In [60]:
# import seaborn as sns
# plt.figure(figsize=(10,10))
# print(cc.std(axis=0).max())
# print(cc.std(axis=0).mean())

In [61]:
# sns.heatmap(((cc[0,:,:]-cc[1,:,:])))

In [62]:
# from helper_funcs import get_triu
# #abs(get_triu((cc[50,:,:]-cc[10,:,:]))).mean()

In [63]:
# ccc = np.array([get_triu(predicted[i,:,:]) for i in range(predicted.shape[0])])

In [64]:
# ratio = ccc.std(axis=0).mean() / c
# f'between sub variability is {ratio:.2f} times larger than within sub variability'

In [65]:
# predicted_samples = np.zeros((100,len(df),51,51))
# predicted_bg_samples = np.zeros((100,len(df),51,51))
# for s in  tqdm(range(len(df))):
#     for idx in range(100):
#         predicted_samples[idx,s,:,:] = predict_cmat(s,patients[s],idx=idx)
#         predicted_bg_samples[idx,s,:,:] = predict_cmat(s,False,idx=idx)

In [24]:
# latent_data = dict()

In [25]:
# latent_data['cData'] = cData

In [26]:
# latent_data['Z_mu'] = Z_mu
# latent_data['Z_sigma'] = Z_sigma
# latent_data['Z'] = Z

# latent_data['S_mu'] = S_mu
# latent_data['S_sigma'] = S_sigma
# latent_data['S'] = S

# latent_data['z_samples'] = z_samples
# latent_data['s_samples'] = s_samples

# latent_data['z_embedding'] = z_embedding
# latent_data['s_embedding'] = s_embedding

# latent_data['predicted'] = predicted
# latent_data['predicted_bg'] = predicted_bg

In [27]:
# analysis_name

'CVAE_2021-10-28 05:12:17.623449'

In [28]:
# ofn = os.path.join('../Data',f'cmat_latents_{analysis_name}.pickle')
# ofn

'../Data/cmat_latents_CVAE_2021-10-28 05:12:17.623449.pickle'

In [29]:
# import pickle
# pickle.dump(obj=latent_data,file=open(ofn,'wb'))